In [ ]:
"""
The following code simulates iVQAGF algorithm on QASM simulator.
iVQAGF algorithm is a variational quantum algorithm for general form of semidefinite programs (SDPs) that we proposed in our paper.
Here, we use iVQAGF to solve random instances of Max-Cut problem casted as an SDP.
In the paper, we report simulation results for N = 8, 16, and 32, where N is the dimension of input Hermitian operators of an SDP.
For writing clean, short and easily executable code, here we fix N = 4. By simply substituting another value of N and with some other minor changes, 
one can obtain results for that N.
"""

In [ ]:
# imports
%%capture
import numpy as np
import cvxpy as cp
import scipy as sc
import networkx as nt
from networkx.generators.random_graphs import erdos_renyi_graph, fast_gnp_random_graph
import pennylane as qml

In [ ]:
# Generate a random graph

# Number of vertices
N = 4
# probability
p = 1
# random graph
g = fast_gnp_random_graph(N, p)
# laplacian of the graph
L = nt.linalg.laplacianmatrix.laplacian_matrix(g)

In [ ]:
# Solve SDP formulation of Max-Cut for the above generated graph using CVXPY package

# Constraints
A = []
b = np.ones(N)

for i in range(N):
  temp = np.zeros((N, N))
  temp[i][i] = 1
  A.append(temp)

# Instantiate a symmetric matrix variable.
X = cp.Variable((N, N), symmetric=True)
# Add positive definite constraint
constraints = [X >> 0]

constraints += [
    cp.trace(A[i]@X) == b[i] for i in range(N)
]

# define an SDP with an objective function and constraints
prob = cp.Problem(cp.Maximize(0.25*cp.trace(L.astype('double')@X)),
                  constraints)

# Solve SDP
prob.solve(solver=cp.CVXOPT)

# Results
print("Optimal value:", prob.value)

In [ ]:
def decompose_into_pauli_strings(M):
  """
  This function expresses an Hermitian operator as a linear combination of Pauli strings

  :param M: Hermitian operator
  """
  coeffs, pauli_string_objs = qml.utils.decompose_hamiltonian(M)
  return qml.Hamiltonian(coeffs, pauli_string_objs)

In [ ]:
# quantum circuit settings
num_wires = 4
num_layers = 4

# quantum device
# can also define the number of shots here
device = qml.device("default.qubit", wires=num_wires)

# def the ansatz
def circuit(param, wire):
  """
  This function instantiates quantum circuit based on templates provided by pennylane

  :param param: the parameters of the circuit
  :wires: number of qubits
  """
  qml.templates.StronglyEntanglingLayers(param, wires=wire)

In [ ]:
# defining C^T, B and choi operators.
C = 0.25*L.todense()
C_trans = C.transpose()
B = np.eye(N)
choi_op = np.zeros((N*N, N*N))
# TODO
for i in range(N):
  choi_op[5*i][5*i] = 1

# trace of the solution X
lam = N

In [ ]:
@qml.qnode(device)
def evalute_exp_wire1(param, M):
  """
  This function evaluates the expectation value of a Hermitian operator on qubits 0 and 1

  :param param: circuit parameters
  :param M: Hermitian operator
  """
  circuit(param, wire=[0, 1])
  M_pauli = decompose_into_pauli_strings(M)
  return qml.expval(M_pauli)
  
@qml.qnode(device)
def evalute_exp_wire2(param, M):
  """
  This function evaluates the expectation value of a Hermitian operator on qubits 2 and 3

  :param param: circuit parameters
  :param M: Hermitian operator
  """
  circuit(param, wire=[2, 3])
  M_pauli = decompose_into_pauli_strings(M)
  return qml.expval(M_pauli)

@qml.qnode(device)
def evalute_exp_joint(param1, param2, M):
  """
  This function evaluates the expectation value of a Hermitian operator on all qubits

  :param param: circuit parameters
  :param M: Hermitian operator
  """
  circuit(param1, wire=[0, 1])
  circuit(param2, wire=[2, 3])
  M_pauli = decompose_into_pauli_strings(M)
  return qml.expval(M_pauli)


def cost_func_ALM_grad_step_and_gradient(param1, param2, mu, **kwargs):
  """
  This function returns updated parameters required for gradient descent, as well as it
  also evaluates full_gradient at current parameter values. 

  :param param1: circuit parameters corresponding to wires 0 and 1
  :param param2: circuit parameters corresponding to wires 2 and 3
  :param mu: trace of y
  """
  gradient_single = qml.grad(evalute_exp_wire1, argnum=0)
  grad_C_trans = gradient_single(param1, C_trans)
  gradient_double = qml.grad(evalute_exp_joint, argnum=0)
  grad_choi_op = gradient_double(param1, param2, choi_op)
  full_gradient = lam*grad_C_trans - lam*mu*grad_choi_op
  updated_param = np.add(param1, 0.1*full_gradient)
  return updated_param, full_gradient

def cost_func(param1, param2, mu):
  """
  This function returns cost function value evaluated at current parameter values.

  :param param1: circuit parameters corresponding to wires 0 and 1
  :param param2: circuit parameters corresponding to wires 2 and 3
  :param mu: trace of y
  """
  exp_C_trans = evalute_exp_wire1(param1, C_trans)
  exp_B = evalute_exp_wire2(param2, B)
  exp_choi = evalute_exp_joint(param1, param2, choi_op)
  return lam*exp_C_trans + mu*exp_B - lam*mu*exp_choi

In [ ]:
# Inner function that performs maximization with respect to params1
def sup_wrt_params1(params2, mu):
  
  # initialize params1
  params1 = np.random.random(qml.templates.StronglyEntanglingLayers.shape(n_layers=4, n_wires=2))

  # store params1
  params1_store = [params1]

  max_iterations = 100
  max_tol = 0.1

  # iterate
  while True:
    params1, full_gradient = cost_func_ALM_grad_step_and_gradient(params1, params2, mu)

    # append to the store
    params1_store.append(params1)

    # check the tolerance
    tol = np.linalg.norm(full_gradient)

    # stopping criterion
    if tol <= max_tol:
      break
    
  # return optimal params
  return params1

In [ ]:
max_outer_iterations = 100
max_outer_tol = 1e-04

# initial params2
params2 = np.random.random(qml.templates.StronglyEntanglingLayers.shape(n_layers=4, n_wires=2))
mu = 0.5

# learning step
learning_step = 0.5

# cost function store
cost_func_store = []

# iterate until convergence
for i in range(max_outer_iterations):
  # call the maximizer
  optimal_params1 = sup_wrt_params1(params2, mu)

  # evaluate partial derivative with respect to params2
  gradient_single = qml.grad(evalute_exp_wire2, argnum=0)
  grad_B = gradient_single(params2, B)
  gradient_double = qml.grad(evalute_exp_joint, argnum=1)
  grad_choi_op = gradient_double(optimal_params1, params2, choi_op)
  full_gradient_params2 = mu*grad_B - lam*mu*grad_choi_op

  # evaluate partial derivative with respect to mu
  exp_B = evalute_exp_wire2(params2, B)
  exp_choi = evalute_exp_joint(optimal_params1, params2, choi_op)
  full_gradient_mu = exp_B - lam*exp_choi

  # Update params2 and mu
  params2 = np.subtract(params2, 0.5*full_gradient_params2)
  mu = mu - 0.5*full_gradient_mu

  # cost function at these parameters
  cost_f = cost_func(optimal_params1, params2, mu)

  # append the current value to the store
  cost_func_store.append(cost_f)

  # print
  print("Cost function:", cost_f)